# Action==>
##  0 ==> No Action
##  1 ==> Up
##  2 ==> Up&DownExit
##  3 ==> DownExit
##  4 ==> Down
##  5 ==> Down&UpExit
##  6 ==> UpExit

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import os
from multiprocessing import  Pool
import gc
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('./EU0520/EURUSD_ASK.csv')

df1=pd.read_csv('./EU0520/EURUSD_BID.csv')

df['Spread']=(df.Open-df1.Open)*10000

In [3]:
df['Gmt time'] = pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S.%f') 

In [4]:
df.rename(columns={'Gmt time':'Date'},inplace=True)

In [5]:
df['Pips']=(df.Close-df.Open)*10000

In [6]:
df.Pips.describe()

count    99598.000000
mean         0.007891
std         15.243122
min       -249.500000
25%         -5.900000
50%          0.000000
75%          5.900000
max        303.200000
Name: Pips, dtype: float64

In [7]:
DF=df.loc[::-1].reset_index(drop = True)

In [8]:
DF['Max']=DF.Close.rolling(window=5).max()

In [9]:
DF['Min']=DF.Close.rolling(window=5).min()

In [10]:
DF['MaxShifted']=DF.Max.shift(periods=1)

In [11]:
DF['MinShifted']=DF.Min.shift(periods=1)

In [12]:
df=DF.loc[::-1].reset_index(drop = True)

In [13]:
df['Gain']=(df.MaxShifted>=df.Max)

In [14]:
df['Loss']=(df.MinShifted<=df.Min)

In [15]:
df['Up']=df.Gain&(df.Pips>0)

In [16]:
df['Down']=df.Loss&(df.Pips<0)

df['UpExit']=~df.Gain&(df.Pips<0)

df['DownExit']=~df.Loss&(df.Pips>0)

In [17]:
df['UpRisk']=(df.Open-df.Low)*10000

In [18]:
df['DownRisk']=(df.High-df.Open)*10000

In [19]:
df.Up=df.Up&(df.UpRisk<df.Pips)

In [20]:
df.Down=df.Down&((-df.Pips)>df.DownRisk)

In [21]:
df[['Up','Down']][:15]

Up   Down
0   False   True
1   False   True
2   False   True
3   False  False
4    True  False
5   False  False
6    True  False
7    True  False
8   False  False
9   False  False
10  False  False
11  False   True
12  False  False
13  False   True
14  False  False

df['Action']=1*((df.Up)&(~df.DownExit)&(~df.Down)&(~df.UpExit))+2*((df.Up)&(df.DownExit)&(~df.Down)&(~df.UpExit))+3*((~df.Up)&(df.DownExit)&(~df.Down)&(~df.UpExit))+4*((~df.Up)&(~df.DownExit)&(df.Down)&(~df.UpExit))+5*((~df.Up)&(~df.DownExit)&(df.Down)&(df.UpExit))+6*((~df.Up)&(~df.DownExit)&(~df.Down)&(df.UpExit))

In [22]:
df['Action']=(df.Up&(~df.Down))*1-((~df.Up)&(~df.Down))*1

In [23]:
df.Action.value_counts()

-1    52978
 1    23485
 0    23135
Name: Action, dtype: int64

In [24]:
col=[df.columns]

In [25]:
col

[Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Pips', 'Max', 'Min',
        'MaxShifted', 'MinShifted', 'Gain', 'Loss', 'Up', 'Down', 'UpRisk',
        'DownRisk', 'Action'],
       dtype='object')]

In [26]:
df.columns=[np.arange(18)]

In [27]:
df[[17]]

17
0      0
1      0
2      0
3     -1
4      1
...   ..
99593 -1
99594 -1
99595 -1
99596 -1
99597 -1

[99598 rows x 1 columns]

In [28]:
def Previousdata(df):
    for i in np.arange(1,12):
        df=pd.concat([df,(df[[1]]).shift(periods=i)],axis=1,ignore_index=True)
        df=pd.concat([df,(df[[2]]).shift(periods=i)],axis=1,ignore_index=True)
        df=pd.concat([df,(df[[3]]).shift(periods=i)],axis=1,ignore_index=True)
        df=pd.concat([df,(df[[4]]).shift(periods=i)],axis=1,ignore_index=True)
    return df

In [29]:
def PARALLELIZE(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [30]:
df=PARALLELIZE(df,Previousdata)

In [32]:
df.columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Pips',
        'Max', 'Min', 'MaxShifted', 'MinShifted', 'Gain', 'Up', 'Loss', 'Down',
         'UpRisk', 'DownRisk', 'Action']+list(map(str,list(range(0,11*4))))

In [33]:
df[73:76][['0','1','2','3','4','5','6','7','Date', 'Open', 'High', 'Low', 'Close']]

0        1        2        3        4        5        6        7  \
73  1.32773  1.32781  1.32540  1.32550  1.32718  1.32811  1.32675  1.32741   
74  1.32547  1.32643  1.32532  1.32636  1.32773  1.32781  1.32540  1.32550   
75  1.32612  1.32767  1.32592  1.32648  1.32547  1.32643  1.32532  1.32636   

                  Date     Open     High      Low    Close  
73 2005-01-06 01:00:00  1.32547  1.32643  1.32532  1.32636  
74 2005-01-06 02:00:00  1.32612  1.32767  1.32592  1.32648  
75 2005-01-06 03:00:00  1.32655  1.32677  1.32544  1.32588

In [34]:
df.to_csv('EURUSD.csv',index=False)